# Basic usage of the package

### Loading files

The `load_from_csv()` function provides a relief of dealing with numerous files. It can load a single CSV file, but it can also scan a directory and load all files in that directory and returns a `Dataset` object. While appending new `Timeseries` to the `Dataset`, it makes sure that timeseries from the same locations and sensors are merged together. It is important to note, that the timeseries should be an uninterupted series of measurements.

*Example:*

*In Project Grow, we had absolute pressure transducers deployed across an agricultural field. We obtained all measurements for sensor altitude necessary for converting the raw measurments to groundwater levels in meters above sea level. However, over time, some piezometers had to be shortened, and the sensors were re-deployed at the same location with the same cable. Hence, we had to split the timeseries into pieces to account for chaning sensor altitude.The loggers we use in Project Grow record temerature and pressure, but there are also other products that can measure electricul conductivity as well. In the CSV file each timeseries is saved in a separate column. `load_from_csv()` function splits the columns into individual timeseries and creates a list of `Timeseries` objects.*

*Note: In the vanessen file, the metadata of the diver and the station it is deployed to is stored in the header of the file. To extract those from all the files, I used regex. It is important that the user knows the naming patterns of the stations and replace the default pattern using a kwarg `location_pattern`. It is possible to add an OR (|) operator to try matching multiple patterns. For us, the following pattern matches all substrings that start with two letters, have two numbers in the middle and end with one letter or include a string 'Barodived' `pattern = r'[A-Za-z]{2}\d{2}[A-Za-z]{1}|Barodiver'`*

In [ ]:
from pathlib import Path

test_data = Path('/mnt/g/My Drive/Kinrooi_FieldData/DiverMate/CSV')

## Explore the Timeseries

In [ ]:
from gensor import read_from_csv

pattern = r'[A-Za-z]{2}\d{2}[A-Za-z]{1}|Barodiver'

data = read_from_csv(path=test_data, 
                    file_format='vanessen', 
                    location_pattern=pattern)


## Compensation

To compensate the measurements from an absolute pressure sensor for atmospheric pressure we need to do two things:
- Identify the atmospheric pressure timeseries
- Include the sensor elevation in the submerged sensor timeseries

In [ ]:
# identify the barometric timeseries in your dataset
baro = data.filter(station='Barodiver', variable='pressure')
baro

In [ ]:
# Adding extra information to the submerged sensor timeseries
pb15a = data.filter(station='PB15A', variable='pressure')
pb15a

In [ ]:
pb15a.sensor_alt = 33.36

In [ ]:
from gensor import compensate

pb15a_comp = compensate(pb15a, baro, drop_low_wc=False)

In [ ]:
pb15a_comp.transform("difference").plot()

In [ ]:
fig, ax = pb15a_comp.detect_outliers("iqr").plot()
